In [1]:
%%bash
set -euxo pipefail
sudo apt-get update -y
sudo apt-get install -y \
  python3.10 python3.10-venv python3.10-dev \
  git ninja-build build-essential \
  libgl1 libglib2.0-0 ffmpeg


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,385 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.2 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,643 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:13 https://ppa.launchpadcontent.n

+ sudo apt-get update -y
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sudo apt-get install -y python3.10 python3.10-venv python3.10-dev git ninja-build build-essential libgl1 libglib2.0-0 ffmpeg
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [2]:
%%bash
set -euxo pipefail
python3.10 -m venv /content/py310
source /content/py310/bin/activate
python -V
pip install -U pip setuptools wheel


Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 KB 14.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.1.2
    Uninstalling setuptools-68.1.2:
      Successfully uninstalled setuptools-68.1.2
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Uninstalling pip-22.0.2:
      Successfully uninstalled pip-22.0.2


+ python3.10 -m venv /content/py310
+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ python -V
+ pip install -U pip setuptools wheel


In [3]:
%%bash
set -euxo pipefail
cd /content
git clone --recurse-submodules https://github.com/DopeMeerkat/TRELLIS.git
cd /content/TRELLIS
git submodule update --init --recursive


Submodule path 'trellis/representations/mesh/flexicubes': checked out '815e075a2a400d06c48d94c347674344ed6ae5c5'


+ cd /content
+ git clone --recurse-submodules https://github.com/DopeMeerkat/TRELLIS.git
Cloning into 'TRELLIS'...
Submodule 'trellis/representations/mesh/flexicubes' (https://github.com/MaxtirError/FlexiCubes.git) registered for path 'trellis/representations/mesh/flexicubes'
Cloning into '/content/TRELLIS/trellis/representations/mesh/flexicubes'...
+ cd /content/TRELLIS
+ git submodule update --init --recursive


In [4]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

# Torch
pip uninstall -y torch torchvision torchaudio || true
pip install --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121

python -c "import torch; print('torch', torch.__version__); print('cuda', torch.version.cuda); print('avail', torch.cuda.is_available()); print('gpu', torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)"


# Build tools + common deps
pip install -U packaging ninja cmake psutil

pip install "kaolin==0.18.0" -f "https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html"

python -c "import kaolin; print('kaolin', kaolin.__version__)"

# flash-attn (T4 is SM75; build for it)
export CUDA_HOME=/usr/local/cuda
export PATH="$CUDA_HOME/bin:$PATH"
export TORCH_CUDA_ARCH_LIST="7.5"
pip uninstall -y flash-attn || true
pip install --no-build-isolation -v flash-attn==2.8.3

python -c "import flash_attn; print('flash_attn', flash_attn.__version__)"


Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 37.1 MB/s  0:00:08
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 158.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 137.2 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 244.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 237.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 39.6 MB/s  0:00:07
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 62.3 MB/s  0:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 181.1 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 126.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 81.6 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 181.4 MB

+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ pip uninstall -y torch torchvision torchaudio
+ pip install --index-url https://download.pytorch.org/whl/cu121 torch==2.5.1+cu121 torchvision==0.20.1+cu121 t

In [5]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate
cd /content/TRELLIS

# prevent extension collisions between runs
rm -rf /tmp/extensions || true

# TRELLIS setup for basic deps + spconv backend
source ./setup.sh --basic --spconv

# nvdiffrast (match your local v0.3.4)
pip install --no-build-isolation "git+https://github.com/NVlabs/nvdiffrast.git@v0.3.4"

python - <<'PY'
import importlib.metadata as md, torch
print("[torch]", torch.__version__)
try:
    import spconv
    print("[spconv] OK")
except Exception as e:
    print("[spconv] FAIL", e)

import nvdiffrast, os
print("[nvdiffrast]", md.version("nvdiffrast"), os.path.dirname(nvdiffrast.__file__))
PY


[SYSTEM] PyTorch Version: 2.5.1+cu121, CUDA Version: 12.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 165.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 134.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 96.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 46.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 42.2 MB/s  0:00:07
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 124.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.6/145.6 MB 149.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 160.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 176.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 35.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 133.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━

+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ cd /content/TRELLIS
+ rm -rf /tmp/extensions
+ source ./setup.sh --basic --spconv
+++ getopt -o h --long help,new-env,basic,train,xformers,flash-attn,diffoct

In [6]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

python - <<'PY'
from pathlib import Path
import re

p = Path("/content/TRELLIS/trellis/pipelines/base.py")
s = p.read_text()

# Ensure os import exists
if not re.search(r"^\s*import os\s*$", s, re.M):
    s = re.sub(r"(?m)^((?:from .+\n|import .+\n)+)", r"\1import os\n", s, count=1)

needle = 'config_file = hf_hub_download(path, "pipeline.json")'
m = re.search(r"(?m)^(?P<indent>\s*)" + re.escape(needle) + r"\s*$", s)
if not m:
    raise SystemExit("Expected line not found: " + needle)

indent = m.group("indent")
replacement = "\n".join([
    f"{indent}# Local folder support: if `path` is a directory, read pipeline.json from disk",
    f"{indent}if os.path.isdir(path):",
    f"{indent}    config_file = os.path.join(path, 'pipeline.json')",
    f"{indent}    if not os.path.exists(config_file):",
    f"{indent}        raise FileNotFoundError('Missing pipeline.json in local folder: ' + str(path))",
    f"{indent}else:",
    f"{indent}    config_file = hf_hub_download(path, 'pipeline.json')",
])

s = re.sub(r"(?m)^" + re.escape(indent) + re.escape(needle) + r"\s*$", replacement, s, count=1)
p.write_text(s)
print("Patched:", p)
PY

/content/py310/bin/python -m py_compile /content/TRELLIS/trellis/pipelines/base.py
echo "base.py compiles"


Patched: /content/TRELLIS/trellis/pipelines/base.py
base.py compiles


+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ python -
+ /content/py310/bin/python -m py_compile /content/TRELLIS/trellis/pipelines/base.py
+ echo 'base.py compiles'


In [7]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

pip install -U huggingface_hub

python - <<'PY'
from huggingface_hub import snapshot_download
path = snapshot_download(
    repo_id="microsoft/TRELLIS-image-large",
    local_dir="/content/TRELLIS-image-large",
    local_dir_use_symlinks=False,
)
print("downloaded to:", path)
PY

# Use local absolute folder
sed -i 's|from_pretrained("./TRELLIS-image-large")|from_pretrained("/content/TRELLIS-image-large")|g' /content/TRELLIS/generate.py


downloaded to: /content/TRELLIS-image-large


+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ pip install -U huggingface_hub
+ python -
/content/py310/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_u

In [8]:
%%bash
set -euxo pipefail
mkdir -p /content/outputs
/content/py310/bin/python /content/TRELLIS/generate.py /content/hambaga.jpg /content/outputs/hambaga


[SPARSE] Backend: spconv, Attention: flash_attn
[SPARSE][CONV] spconv algo: native
[ATTENTION] Using backend: flash_attn
3D generation completed successfully!
Generated 1 Gaussian splat(s)
Generated 1 radiance field(s)
Generated 1 mesh(es)
Saved raw mesh OBJ: /content/outputs/hambaga_raw.obj
Done. Note: Rendering videos is omitted to keep the script lightweight for Colab.


+ mkdir -p /content/outputs
+ /content/py310/bin/python /content/TRELLIS/generate.py /content/hambaga.jpg /content/outputs/hambaga
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:97: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:163: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:243: FutureWarn

## API SECTION


In [41]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate
pip install -U trimesh

+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ pip install -U trimesh


In [10]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

pip install -U fastapi uvicorn python-multipart pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 111.0 MB/s  0:00:00



+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ pip install -U fastapi uvicorn python-multipart pyngrok


In [33]:
%%bash
set -euxo pipefail

wget -q -O /tmp/ngrok.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
tar -xzf /tmp/ngrok.tgz -C /tmp

sudo mv /tmp/ngrok /usr/local/bin/ngrok
sudo chmod +x /usr/local/bin/ngrok

ngrok version
which ngrok

ngrok version 3.36.1
/usr/local/bin/ngrok


+ wget -q -O /tmp/ngrok.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
+ tar -xzf /tmp/ngrok.tgz -C /tmp
+ sudo mv /tmp/ngrok /usr/local/bin/ngrok
+ sudo chmod +x /usr/local/bin/ngrok
+ ngrok version
+ which ngrok


In [48]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

export TRELLIS_MODEL_DIR="${TRELLIS_MODEL_DIR:-/content/TRELLIS-image-large}"
export PYTHONPATH="/content/TRELLIS:${PYTHONPATH:-}"

# Kill whatever is listening on 8000
PID_ON_8000="$(ss -ltnp | awk '/:8000/ {print $NF}' | sed -n 's/.*pid=\([0-9]\+\).*/\1/p' | head -n 1 || true)"
if [ -n "$PID_ON_8000" ]; then kill -9 "$PID_ON_8000" || true; fi
rm -rf /content/__pycache__ || true

nohup /content/py310/bin/python -m uvicorn trellis_api_server:app \
  --app-dir /content \
  --host 0.0.0.0 \
  --port 8000 \
  --log-level info \
  > /content/uvicorn.log 2>&1 &

echo $! > /content/uvicorn.pid

for i in $(seq 1 60); do
  if curl -fsS http://127.0.0.1:8000/health >/dev/null 2>&1; then
    curl -fsS http://127.0.0.1:8000/health
    echo
    echo "Server up. PID=$(cat /content/uvicorn.pid)"
    exit 0
  fi
  sleep 1
done

echo "Server failed to start. Log:"
tail -n 200 /content/uvicorn.log || true
exit 1

{"ok":true,"model_loaded":false,"mode":"raw_mesh_obj","endpoints":["/health","/generate-raw-obj","/generate-obj"]}
Server up. PID=18734


+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ export TRELLIS_MODEL_DIR=/content/TRELLIS-image-large
+ TRELLIS_MODEL_DIR=/content/TRELLIS-image-large
+ export PYTHONPATH=/content/TRELLIS:/env/python
+ PYT

In [45]:
%%bash
set -euxo pipefail

TOKEN="$(tr -d '[:space:]' < /content/ngrok_authtoken.txt)"
ngrok config add-authtoken "$TOKEN"

# kill old ngrok
pkill -f "ngrok http" || true

# IMPORTANT: use 127.0.0.1 instead of localhost to avoid IPv6 [::1]
nohup ngrok http http://127.0.0.1:8000 --log=stdout > /content/ngrok.log 2>&1 &
echo $! > /content/ngrok.pid

sleep 2
echo "== ngrok public URL =="
grep -oE 'https://[a-zA-Z0-9.-]+\.ngrok-free\.(dev|app)' -m 1 /content/ngrok.log || true
tail -n 30 /content/ngrok.log

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
== ngrok public URL ==
https://jeannetta-unreplete-dowdily.ngrok-free.dev
t=2026-03-02T01:01:55+0000 lvl=info msg="no configuration paths supplied"
t=2026-03-02T01:01:55+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2026-03-02T01:01:55+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2026-03-02T01:01:55+0000 lvl=info msg="FIPS 140 mode" enabled=false
t=2026-03-02T01:01:55+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2026-03-02T01:01:56+0000 lvl=info msg="client session established" obj=tunnels.session
t=2026-03-02T01:01:56+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2026-03-02T01:01:57+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://127.0.0.1:8000 url=https://jeannetta-unreplete-dowdily.ngrok-free.dev


++ tr -d '[:space:]'
+ TOKEN=3AMkbjJGvvLTEvEFedqvrGJMXJZ_6SVsSVcrgRQFuxTN3F23o
+ ngrok config add-authtoken 3AMkbjJGvvLTEvEFedqvrGJMXJZ_6SVsSVcrgRQFuxTN3F23o
+ pkill -f 'ngrok http'
+ echo 18313
+ nohup ngrok http http://127.0.0.1:8000 --log=stdout
+ sleep 2
+ echo '== ngrok public URL =='
+ grep -oE 'https://[a-zA-Z0-9.-]+\.ngrok-free\.(dev|app)' -m 1 /content/ngrok.log
+ tail -n 30 /content/ngrok.log


In [44]:
%%bash
set -euxo pipefail
tail -n 200 /content/uvicorn.log

INFO:     Started server process [17726]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
[SPARSE] Backend: spconv, Attention: flash_attn
INFO:     127.0.0.1:32784 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:32800 - "GET /health HTTP/1.1" 200 OK
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:97: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:163: FutureWarning: `torch.cuda.amp.c

+ tail -n 200 /content/uvicorn.log
